#**Importações**

**Bibliotecas**


In [ ]:
pip install pandera

In [ ]:
from google.cloud import storage
import pandera as pa
import pandas as pd
import numpy as np
import os

**Importando DataFrame**

In [ ]:
df = pd.read_csv(r'https://storage.googleapis.com/projetofinalgrupo8/saida/youtube_data_base.csv')

In [ ]:
df.head(2)

,video_id,title,publishedAt,channelId,channelTitle,trending_date,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,country,json_title
0,s9FH4rDMvds,LEVEI UM FORA? FINGI ESTAR APAIXONADO POR ELA!,2020-08-11T22:21:49Z,UCGfBwrCoi9ZJjKiUK8MmJNw,Pietro Guedes,2020-08-12T00:00:00Z,263835,85095,487,4500,False,False,BR,People & Blogs
1,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,UCaO6TYtlC8U5ttz62hTrZgg,JYP Entertainment,2020-08-12T00:00:00Z,6000070,714310,15176,31040,False,False,BR,Music


#**Exploração dos dados**

**Verificando dimensão do DataFrame**

In [ ]:
df.shape

(1021622, 14)

**Tipos dos dados**

In [ ]:
df.dtypes

video_id             object
title                object
publishedAt          object
channelId            object
channelTitle         object
trending_date        object
view_count            int64
likes                 int64
dislikes              int64
comment_count         int64
comments_disabled      bool
ratings_disabled       bool
country              object
json_title           object
dtype: object

#Backup

In [ ]:
df2 = df.copy()

#Renomeando colunas e dados Categoricos

**Criando listas para renomear colunas**

In [ ]:
col_old = ['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle',
       'trending_date', 'view_count', 'likes', 'dislikes', 'comment_count',
       'comments_disabled', 'ratings_disabled', 'country', 'json_title']

col_new = ['id_video', 'titulo_video', 'publicado_em', 'id_canal', 'nome_canal',
           'data_destaque', 'cont_visualizacao', 'curtidas', 'nao_curtidas', 'cont_comentarios',
           'comentarios_desabilitados', 'curtidas_desabilitadas', 'pais', 'categoria']

In [ ]:
df2.columns = col_new

In [ ]:
df2.head(2)

,id_video,titulo_video,publicado_em,id_canal,nome_canal,data_destaque,cont_visualizacao,curtidas,nao_curtidas,cont_comentarios,comentarios_desabilitados,curtidas_desabilitadas,pais,categoria
0,s9FH4rDMvds,LEVEI UM FORA? FINGI ESTAR APAIXONADO POR ELA!,2020-08-11T22:21:49Z,UCGfBwrCoi9ZJjKiUK8MmJNw,Pietro Guedes,2020-08-12T00:00:00Z,263835,85095,487,4500,False,False,BR,People & Blogs
1,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,UCaO6TYtlC8U5ttz62hTrZgg,JYP Entertainment,2020-08-12T00:00:00Z,6000070,714310,15176,31040,False,False,BR,Music


**Vericando tipos de categorias**

In [ ]:
df2['categoria'].unique()

array(['People & Blogs', 'Music', 'Gaming', 'Comedy', 'Sports',
       'Entertainment', 'Education', 'Autos & Vehicles', 'Howto & Style',
       'News & Politics', 'Science & Technology', 'Film & Animation',
       'Travel & Events', 'Pets & Animals', nan], dtype=object)

**Criando listas para renomear as categorias**

In [ ]:
categoria_antiga = ['People & Blogs', 'Music', 'Gaming', 'Comedy', 'Sports',
       'Entertainment', 'Education', 'Autos & Vehicles', 'Howto & Style',
       'News & Politics', 'Science & Technology', 'Film & Animation',
       'Travel & Events', 'Pets & Animals']

categoria_nova = ['pessoas_e_blogs', 'musica', 'jogos', 'comedia', 'esportes',
                  'entretenimento', 'educacao', 'automoveis', 'como_faz_e_estilos',
                  'noticias_e_politicas', 'ciencias_e_tecnologia', 'filme_e_animacao',
                  'viagens_e_eventos', 'animais']

**Renomeando categorias**

In [ ]:
df2['categoria'] = df2['categoria'].replace(categoria_antiga, categoria_nova)

#Formatando datetime

**Verificando os tipos de dados do DataFrame**

In [ ]:
df2.dtypes

id_video                     object
titulo_video                 object
publicado_em                 object
id_canal                     object
nome_canal                   object
data_destaque                object
cont_visualizacao             int64
curtidas                      int64
nao_curtidas                  int64
cont_comentarios              int64
comentarios_desabilitados      bool
curtidas_desabilitadas         bool
pais                         object
categoria                    object
dtype: object

In [ ]:
df2['publicado_em'] = pd.to_datetime(df2['publicado_em']).dt.strftime("%Y-%m-%d %H:%M:%S")
df2['publicado_em'] = pd.to_datetime(df2['publicado_em'])

In [ ]:
df2['data_destaque'] = pd.to_datetime(df2['data_destaque']).dt.strftime("%Y-%m-%d")
df2['data_destaque'] = pd.to_datetime(df2['data_destaque'])

In [ ]:
df2.dtypes

id_video                             object
titulo_video                         object
publicado_em                 datetime64[ns]
id_canal                             object
nome_canal                           object
data_destaque                datetime64[ns]
cont_visualizacao                     int64
curtidas                              int64
nao_curtidas                          int64
cont_comentarios                      int64
comentarios_desabilitados              bool
curtidas_desabilitadas                 bool
pais                                 object
categoria                            object
dtype: object

In [ ]:
df2.sample()

,id_video,titulo_video,publicado_em,id_canal,nome_canal,data_destaque,cont_visualizacao,curtidas,nao_curtidas,cont_comentarios,comentarios_desabilitados,curtidas_desabilitadas,pais,categoria
200903,jWyGWqxAras,Linke Hausbesetzer-Szene (2/2): Die Räumung vo...,2020-10-13 10:00:10,UC1w6pNGiiLdZgyNpXUnA4Zw,DER SPIEGEL,2020-10-17,544677,9292,513,1504,False,False,DE,noticias_e_politicas


#Tratando dados nulos

**Verificando números de dados nulos**

In [ ]:
df2.isna().sum()

id_video                        0
titulo_video                    0
publicado_em                    0
id_canal                        0
nome_canal                      1
data_destaque                   0
cont_visualizacao               0
curtidas                        0
nao_curtidas                    0
cont_comentarios                0
comentarios_desabilitados       0
curtidas_desabilitadas          0
pais                            0
categoria                    2951
dtype: int64

**Tratando os valores nulos da coluna "Categoria"**

In [ ]:
categoria_antiga = ['People & Blogs', 'Music', 'Gaming', 'Comedy', 'Sports',
       'Entertainment', 'Education', 'Autos & Vehicles', 'Howto & Style',
       'News & Politics', 'Science & Technology', 'Film & Animation',
       'Travel & Events', 'Pets & Animals', np.nan]

categoria_nova = ['pessoas_e_blogs', 'musica', 'jogos', 'comedia', 'esportes',
                  'entretenimento', 'educacao', 'automoveis', 'como_faz_e_estilos',
                  'noticias_e_politicas', 'ciencias_e_tecnologia', 'filme_e_animacao',
                  'viagens_e_eventos', 'animais', 'outros']

In [ ]:
df2['categoria'] = df2['categoria'].replace(categoria_antiga, categoria_nova)

In [ ]:
df2.isna().sum()

id_video                     0
titulo_video                 0
publicado_em                 0
id_canal                     0
nome_canal                   1
data_destaque                0
cont_visualizacao            0
curtidas                     0
nao_curtidas                 0
cont_comentarios             0
comentarios_desabilitados    0
curtidas_desabilitadas       0
pais                         0
categoria                    0
dtype: int64

**Verificando nomes de canais com valor nulo**

In [ ]:
df2[df2['nome_canal'].isnull()]

,id_video,titulo_video,publicado_em,id_canal,nome_canal,data_destaque,cont_visualizacao,curtidas,nao_curtidas,cont_comentarios,comentarios_desabilitados,curtidas_desabilitadas,pais,categoria
499155,9b9MovPPewk,Kala Official Teaser | Tovino Thomas | Rohith ...,2021-01-21 12:30:29,UCDQt0y-FCJLRdwhNLutPFZA,NaN,2021-01-23,1008638,72772,1451,3023,False,False,IN,pessoas_e_blogs


**Removendo linha com nome de canal nulo**

In [ ]:
df2.drop(499155, inplace = True)

In [ ]:
df2.isna().sum()

id_video                     0
titulo_video                 0
publicado_em                 0
id_canal                     0
nome_canal                   0
data_destaque                0
cont_visualizacao            0
curtidas                     0
nao_curtidas                 0
cont_comentarios             0
comentarios_desabilitados    0
curtidas_desabilitadas       0
pais                         0
categoria                    0
dtype: int64

#**Validação dos dados**

In [ ]:
df2.dtypes

id_video                             object
titulo_video                         object
publicado_em                 datetime64[ns]
id_canal                             object
nome_canal                           object
data_destaque                datetime64[ns]
cont_visualizacao                     int64
curtidas                              int64
nao_curtidas                          int64
cont_comentarios                      int64
comentarios_desabilitados              bool
curtidas_desabilitadas                 bool
pais                                 object
categoria                            object
dtype: object

In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        "id_video":pa.Column(pa.String),
        "titulo_video":pa.Column(pa.String),
        "publicado_em":pa.Column(pa.DateTime),
        "id_canal":pa.Column(pa.String),
        "nome_canal":pa.Column(pa.String),
        "data_destaque":pa.Column(pa.DateTime),
        "cont_visualizacao":pa.Column(pa.Int),
        "curtidas":pa.Column(pa.Int),
        "nao_curtidas":pa.Column(pa.Int),
        "cont_comentarios":pa.Column(pa.Int),
        "comentarios_desabilitados":pa.Column(pa.Bool),
        "curtidas_desabilitadas":pa.Column(pa.Bool),
        "pais":pa.Column(pa.String),
        "categoria":pa.Column(pa.String)
    }
)

In [ ]:
schema.validate(df2)

#**Informações gerais sobre os tipo de dados e quantidades de não nulos**

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1021621 entries, 0 to 1021621
Data columns (total 14 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   id_video                   1021621 non-null  object        
 1   titulo_video               1021621 non-null  object        
 2   publicado_em               1021621 non-null  datetime64[ns]
 3   id_canal                   1021621 non-null  object        
 4   nome_canal                 1021621 non-null  object        
 5   data_destaque              1021621 non-null  datetime64[ns]
 6   cont_visualizacao          1021621 non-null  int64         
 7   curtidas                   1021621 non-null  int64         
 8   nao_curtidas               1021621 non-null  int64         
 9   cont_comentarios           1021621 non-null  int64         
 10  comentarios_desabilitados  1021621 non-null  bool          
 11  curtidas_desabilitadas     1021621 no

#Exportando DataFrame para o GCP

In [ ]:
from google.cloud import storage
import os
serviceAccount = '/content/projetofinalgrupo8-2dcd866c3f46.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount


client = storage.Client()
bucket = client.get_bucket('projetofinalgrupo8')
    
bucket.blob('saida/youtube_tratado_pandas.csv').upload_from_string(df2.to_csv(index=False), 'text/csv')